![Logo 1](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech1.jpg)
<div class="alert alert-block alert-info">
<h1> Praktyczne zastosowania chmury obliczeniowej </h1>
<h2> 4.2. <i>Kompleksowa publiczna chmura obliczeniowa - cz. 2</i>  [laboratoria]</h2> 
<h3> Patryk Żywica (2021)</h3>
</div>

![Logo 2](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech2.jpg)

# Kompleksowa publiczna chmura obliczeniowa - cz. 2

Niniejszy moduł przedstawia podstawowe usługi AWS oraz metody ich konfiguracji. Omawiane treści wymagają ogólnej wiedzy na temat usług AWS oraz modelu IaaS.

## Wirtualna sieć prywatna (VPC)

AWS oferuje usługę wirtualnej sieci prywatnej pod nazwą Virtual Private Cloud (VPC). VPC to logicznie wydzielona część chmury AWS, zlokalizowana w pojedynczym regionie. W ramach VPC należy następnie zdefiniować podsieci, które są przypisane do konkretnego AZ. W przypadku AWS to właśnie usługa VPC umożliwia przydzielanie zasobów do fizycznych lokalizacji oraz ich separację. Dany zasób jest lokowany na podstawie wirtualnej sieci prywatnej oraz podsieci, do której został przydzielony.

W przypadku VPC dostępne jest od /16 do /28 adresów. Poniższe 5 adresów zawsze wyłączone z użytku: 
- 10.0.0.0 (adres sieci)
- 10.0.0.1 (wewnętrzna komunikacja)
- 10.0.0.2 (DNS)
- 10.0.0.3 (future)
- 10.0.0.255 (broadcast)

W VPC każda maszyna dostaje prywatny adres z jednej podsieci. Publiczny adres IP przydzielany jest tylko wtedy, gdy podsieć tego wymaga lub poprzez Elastic IP. Elastic IP może być przypisany do dowolnej maszyny wirtualnej EC2.


Poprawna konfiguracja VPC wymaga też zdefiniowana tablicy routingu, domyślnie zapewniona komunikacja wewnątrz VPC (lub podsieci). Pozostałe ustawienia (np. dostęp do Internetu) należy skonfigurować samodzielnie.

Początkowo w każdym regionie zdefiniowane jest domyślne VPC, które standardowo przypisuje publiczne adresy IP maszyną wirtualnym. Domyślne VPC powinno być używane tylko do eksperymentowania z nowymi usługami. Ponadto, ponieważ konfiguracja VPC jest stosunkowo trudna, szczególnie dla początkujących użytkowników, AWS udostępnia prosty kreator pozwalający zbudować nowe VPC w oparciu o jeden z czterech szablonów.

Ruch sieciowy w ramach VPC można ograniczyć korzystając z dwóch mechanizmów:
- security groups: przypisane do konkretnej maszyny wirtualnej, określa tylko reguły incoming (domyślnie wszystko deny), jest to zapora sieciowa typu stateful
- ACL: przypisane na poziomie podsieci, domyślnie dopuszcza cały ruch w obie strony), jest to rozwiązanie typu stateless

## Elastic Cloud Compute (EC2) - maszyny wirtualne

Maszyny wirtualne EC2 stanowią bazę dla niemal wszystkich usług obliczeniowych AWS w szczególności dla:
* EC2 autoscaling: automatyzacja alokacji nowych maszyn wirtualnych
* Elastic Container Registry: repozytorium obrazów Docker
* Elastic Container Service: środowisko uruchomieniowe dla Dockera
* Elastic Beanstalk: platforma w modelu PaaS
* Lambda: bezstanowe obliczenia w chmurze (FaaS)
* EKS: zarządzane środowisko Kubernetes
* Lightsail: uproszczone tworzenie predefiniowanych serwisów
* Batch: przetwarzanie wsadowe z wykorzystaniem EC2
* Fargate: kontenery bez konieczności alokowania zasobów

Tworzenie maszyny wirtualnej w AWS daje bardzo wiele możliwości konfiguracji. Dostępne opcje pogrupowane są w 9 kategorii:
1. AMI (Amazon Machine Image): szablon maszyny wirtualnej definiującej system operacyjny i oprogramowanie. Można tworzyć własne AMI na bazie istniejących obrazów (AMI jest zdefiniowane per region)
1. Typ maszyny wirtualnej: 
 * general purpose: a1, m4, m5, t2, t3
 * compute: c4, c5
 * memory: r4, r5, x1, z1
 * storage: d2, h1, i3
 * accelerated computing (gpu, fpga): f1, g3, g4, p2, p3

	Nazewnictwo typów jest bardzo jednorodne, przykładowo: `t3.large`:
	* t: nazwa rodziny maszyn, 
	* 3: generacja
	* large: rozmiar maszyny

1. Sieć: wybieramy VPC, podsieć, określamy, czy zostanie przydzielony publiczny IP, domyślne VPC dla uproszczenia zawsze przydziela publiczne IP
1. IAM role: zdefiniowanie IAM roles, które może przyjmować VM co umożliwia jej dostęp do innych zasobów AWS
1. User data: miejsce na konfigurację `cloud-init`
1. Przechowywanie danych: standardowo maszyny EC2 nie maja dostępnego dysku. Zaleca się skonfigurowanie dwóch wolumenów dla każdej maszyny: root volume (dla systemu operacyjnego) oraz opcjonalnie dysku na dane. Dostępne mamy dwa rodzaje dysków
	* EBS (domyślnie), trwałe przechowywanie danych
	* instance store: tymczasowy dysk, fizycznie na tym samym serwerze co VM
1. Tagi: najczęściej wykorzystywany jest tag `name`
1. Security groups: firewall danej maszyny wirtualnej
1. Klucze SSH: określenie kluczy dostępowych SSH do maszyny wirtualnej

W zależności od tego czy maszyna wirtualna EC2 ma podpięty trwały dysk, czy też nie, może ona przechodzić pomiędzy różnymi stanami:

1. dla maszyn bez trwałego magazynu danych: 

	> pending -> running -> shutting-down -> terminated

1.  dla maszyn z trwałym magazynem danych:

	> pending -> running -> stopping -> stopped -> pending -> running ...
	 
	 
Za EC2 w stanie `stopped` nie są naliczane opłaty. Ponadto, jeśli maszyna jest restartowana, zatrzymuje ten sam adres IP i internal storage. W przypadku wyłączenia i ponownego włączanie nie jest to możliwe.

Jeśli istnieje potrzeba otrzymania informacji o danej instancji EC2 z wnętrza maszyny wirtualnej (np. na potrzeby skryptu konfigurującego oprogramowanie), jest ona dostępna za pomocną protokołu HTTP pod prywatnym adresem `169.254.169.254/latest/meta-data`. Informacje obejmują:
* nazwę hosta
* adresy IP
* region i AZ

## Przechowywanie danych

Zasadniczo AWS oferuje dwa rodzaje usług do przechowywania danych:
* **blokowe**: szybkie, zużywające mniej zasobów sieciowych, ale znacznie droższe. Dyski blokowe pozwalają na modyfikację części plików i dzięki temu nadają się do instalacji oprogramowania czy systemu operacyjnego. Usługi EBS oraz EFS.
* **obiektowe**: wolniejsze, każda zmiana wymaga nadpisania całego pliku. Usługa S3 i jej warianty (np. Glacier).

Praktycznie każda maszyna wirtualna wymaga dysku, na którym powinien znajdować się system operacyjny. Elastic Block Storage (EBS) jest usługą, która oferuje wolumeny sieciowe o różnych parametrach. Wszystkie oferują automatyczna replikacje w ramach danego AZ, możliwość eksportu migawek do S3 (można przechowywać w innym regionie dla zwiększonego bezpieczeństwa) oraz opcjonalne szyfrowanie danych. Należy pamiętać, że wolumeny EBS są nie zależne od EC2 i należy je usuwać osobno.

Dwa główne typy EBS:
* **ssd**: general purpose (16k IOPS, 250MiB/s) i provinsioned IOPS (64k IOPS, 1GiB/s)
* **hdd**: throughput-optimized(500IOPS, 500MiB/s) i cold (250IOPS, 250MiB/s)

Dyski blokowe nie są optymalnym rozwiązaniem do przechowywania dużych ilości rzadko modyfikowanych danych. W tym celu lepiej skorzystać z usługi Simple Storage Service (S3). Usługa ta pozwala na przechowywanie i zarządzanie całymi plikami. W ramach S3 można wyróżnić kilka wariantów pozwalających dopasować usługę do potrzeb:
* Standard, Standard Infrequent Access: do ogólnych zastosowań, wysoce bezpieczne, przynajmniej 3 kopie w różnych AZ, infrequent access do rzadszego dostępu, ale ciągle bardzo szybkiego
* One-Zone: kopie przechowywane tylko w jednej AZ oraz jest ich mniej
* Intelligent-Tiering: dodatkowa oplata za obiekt, system automatycznie szuka nie używanych obiektów i przenosi je do infrequent access
* Glacier: dostęp do danych od minuty do godziny, przechowywanie rzadko używanych danych, bez potrzeby nagłego dostępu
* Glacier Deep Archive: dostęp do danych w 12h, dla danych używanych 1-2 razy w roku

Opłaty:
* za zajmowaną przestrzeń (GB)
* za transfer OUT region (GB)
* za liczbę żądań, GET ma inną stawkę niż pozostałe
* za każdy obiekt dla Intelligent Tiering
* za odczytywanie danych dla Infrequent Access i Glacier


# Learner Labs

Zajęcia będą realizowane z wykorzystaniem narzędzia Learner Labs oferowanego w ramach inicjatywy AWS Academy. Każdy uczestnik zajęć dysponuje pulą $100 do wykorzystania w ramach zajęć. Learner Labs pozwala na korzystanie z infrastruktury AWS w dowolny sposób, pod warunkiem, że jest to związane z realizacją zajęć dydaktycznych. Ponadto obowiązuje kilka ograniczeń:
 * pojedyncza sesja jest ograniczona czasowo do 4 godzin, istniejącą sesję można przedłużyć wybierając ponownie opcję `Start`
 * zasoby utworzone podczas sesji, nie są automatycznie usuwane po jej zakończeniu (choć mogą zostać usunięte/wyłączone), zatem obowiązkiem studenta jest odpowiednie zarządzanie zasobami
 * wszystkie zasoby muszą być utworzone w regionie `us-east-1`, choć jest to ustawienie domyślne zawsze warto sprawdzić ustawienie regionu szczególnie gdy otrzymujemy błąd mówiący o braku uprawnień
 * możliwe jest utworzenie maksymalnie 9 maszyn wirtualnych EC2 korzystających łącznie z co najwyżej 32 vCPU, brak dostępu do maszyn z GPU, próba przekroczenia 20 maszyn łącznie powoduje bezpowrotne zablokowanie/usunięcie konta AWS
 * maksymalna pojemność wolumenów dyskowych to 100GB
 * konfiguracja usługi IAM jest bardzo silnie ograniczona, należy korzystać z domyślnej roli `LabRole`
 * dostępne są następujące usługi:
    * Amazon API Gateway
    * AWS App Mesh
    * Application Auto Scaling
    * AWS AppSync
    * Amazon Athena
    * Amazon Aurora
    * AWS Backup
    * AWS Batch
    * AWS Certificate Manager (ACM)
    * AWS Cloud9
    * AWS CloudFormation
    * Amazon CloudFront
    * Amazon CloudSearch
    * AWS CloudShell
    * AWS CloudTrail
    * Amazon CloudWatch
    * AWS CodeCommit
    * AWS CodeDeploy
    * Amazon Comprehend
    * AWS Config
    * AWS Cost and Usage Report
    * AWS Cost Explorer
    * AWS Data Pipeline
    * AWS DeepComposer
    * AWS DeepLens
    * AWS DeepRacer
    * AWS Directory Service
    * Amazon DynamoDB
    * Amazon EC2 Auto Scaling
    * AWS Elastic Beanstalk
    * Amazon Elastic Block Store (EBS)
    * Amazon Elastic Compute Cloud (EC2)
    * Amazon Elastic Container Registry (ECR)
    * Amazon Elastic Container Service (ECS)
    * Amazon Elastic File System (EFS)
    * Amazon Elastic Inference
    * Amazon Elastic Kubernetes Service (EKS)
    * Elastic Load Balancing (ELB)
    * Amazon Elastic MapReduce (EMR)
    * Amazon ElastiCache
    * Amazon EventBridge
    * AWS Fargate
    * Amazon Forecast
    * AWS Glue
    * AWS Glue DataBrew
    * Amazon GuardDuty
    * AWS Health
    * AWS Identity and Access Management (IAM)
    * AWS IAM Access Analyzer
    * Amazon Inspector
    * AWS IoT 1-Click
    * AWS IoT Analytics
    * AWS IoT Core
    * AWS IoT Greengrass
    * Amazon Kendra
    * AWS Key Management Service (KMS)
    * Amazon Kinesis
    * Amazon Kinesis Video Streams
    * AWS Lambda
    * Amazon Lex
    * Amazon Machine Learning (Amazon ML)
    * AWS Marketplace Subscriptions
    * AWS Mobile Hub
    * Amazon Neptune
    * AWS OpsWorks
    * Amazon Personalize
    * Amazon Polly
    * Amazon QuickSight
    * Amazon Redshift
    * Amazon Rekognition
    * Amazon Relational Database Service (RDS)
    * AWS Resource Groups & Tag Editor
    * AWS RoboMaker
    * Amazon Route 53
    * Amazon SageMaker
    * AWS Secrets Manager
    * AWS Security Hub
    * AWS Security Token Service (STS)
    * AWS Serverless Application Repository (SAR)
    * AWS Service Catalog
    * Amazon Simple Notification Service (SNS)
    * Amazon Simple Queue Service (SQS)
    * Amazon Simple Storage Service (S3)
    * Amazon Simple Storage Service Glacier (S3 Glacier)
    * Amazon Simple Workflow Service (SWF)
    * AWS Step Functions
    * AWS Storage Gateway
    * AWS Systems Manager (SSM)
    * Amazon Textract
    * Amazon Timestream
    * Amazon Transcribe
    * Amazon Translate
    * AWS Trusted Advisor
    * Amazon Virtual Private Cloud (Amazon VPC)
    * AWS WAF - Web Application Firewall
    * AWS Well-Architected Tool
    * AWS X-Ray

Po zalogowaniu się do Learner Lab dostępne jest kilka kluczowych opcji.

<img src="obrazy/learner-lab.png" width="75%" alt="Widok głównego ekranu Learner Labs">

1. Monitorowanie zużytych środków. Należy pamiętać, że część kosztów naliczana może być z opóźnieniem do 12 godzin.
1. Przycisk rozpoczynający laboratorium. Jego ponowne wybranie pozwala przedłużyć obecną sesję o kolejne 4 godziny.
1. Przycisk dostępowy do AWS Web Console, zielona ikonka oznacza, że środowisko zostało poprawnie zainicjalizowane.
1. W środkowej części znajduje się konsola Linuxowa, oferująca podstawowe narzędzia (np. ssh wraz z kluczami) oraz dostęp do narzędzia AWS CLI

# Zadania do realizacji na zajęciach


## Automatyczne skalowanie instancji

Skonfiguruj mechanizm automatycznego skalowania tak aby zarządzał pulą maszyn wirtualnych (czysty Amazon Linux). Domyślnie w puli powinna być jedna instancja, z możliwością zwiększenia do 2 maszyn. Jako mechanizm śledzenia obciążenia wybierz obciążenie procesora.

Gdy pula zostanie już zainicjalizowana, zaloguj się poprzez SSH na pierwszą maszynę wirtualną i uruchom:

```bash
sudo amazon-linux-extras install epel -y
sudo yum install stress -y
stress --cpu 8 --io 4 --vm 2 --vm-bytes 128M --timeout 600s
```

Obserwuj statystyki puli automatycznego skalowania, po pewnym czasie uruchomiona zostanie automatycznie kolejna maszyna. Odczekaj 10 minut i sprawdź, czy pula zostanie automatycznie zmniejszona do jednej instancji, gdy stress test dobiegnie końca.